### Cell map of the zebrafish embryo using graph abstraction

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.sparse import csr_matrix
import scanpy as sc

sc.set_figure_params(dpi=200, color_map='viridis')  # low dpi (dots per inch) yields small inline figures
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
results_file = './write/zebrafish_theis.sh5ad'

scanpy==1.5.1 anndata==0.7.3 umap==0.4.4 numpy==1.19.0 scipy==1.5.0 pandas==1.0.5 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.2 louvain==0.7.0 leidenalg==0.8.1


In [15]:
var = pd.read_csv('/Users/reang/Desktop/reang/compbio/datasets/data/gene_names.txt', index_col=0, header=None)

In [16]:
cluster_indices = np.genfromtxt('/Users/reang/Desktop/reang/compbio/datasets/data/cell_cluster_indexes.txt', dtype=int)
clusters = pd.Categorical(cluster_indices)
clusters.categories = ['?'] + np.genfromtxt('/Users/reang/Desktop/reang/compbio/datasets/data/cluster_names.txt', dtype=str, delimiter=',').tolist()
obs = pd.DataFrame({'clusters': clusters})

In [17]:
cell_cell_edges = np.genfromtxt('/Users/reang/Desktop/reang/compbio/datasets/data/cell_cell_edges.csv', dtype=int, delimiter=',')
cell_cell_edges -= 1  # our indexing starts with 0 as common in Python, C etc.
from scipy.sparse import coo_matrix
rows = cell_cell_edges[:, 0]
cols = cell_cell_edges[:, 1]
length = len(obs)
ones = np.ones(len(rows), np.uint32)
connectivities = coo_matrix((ones, (rows, cols)), shape=(length, length))
# make sure it's symmsetric
connectivities = connectivities + connectivities.T

In [18]:
uns = {'neighbors': {'connectivities': connectivities.tocsr()}}

In [19]:
adata = sc.AnnData(obs=obs, var=var, uns=uns)
# remove the outliers
adata = adata[adata.obs['clusters'] != '?']

In [20]:
adata

View of AnnData object with n_obs × n_vars = 53181 × 30677
    obs: 'clusters'
    uns: 'neighbors'
    obsp: 'connectivities'

In [25]:
adata.write(results_file)

TypeError: 'NoneType' object is not subscriptable

In [26]:
adata = sc.read(results_file)

ValueError: Reading with filekey 'write/zebrafish_theis.sh5ad' failed, the inferred filename PosixPath('write/write/zebrafish_theis.sh5ad.h5ad') does not exist. If you intended to provide a filename, either use a filename ending on one of the available extensions {'loom', 'h5', 'mtx.gz', 'data', 'anndata', 'h5ad', 'tsv', 'tab', 'soft.gz', 'xlsx', 'csv', 'mtx', 'txt'} or pass the parameter `ext`.

In [27]:
sc.tl.umap(adata)

computing UMAP


KeyError: 'connectivities_key'